# Лекция 3. Векторизация на практике

In [1]:
!pip install datasets transformers tokenizers -q

## Прочитаем датасет

Работать будем с [датасетом Сбера](https://huggingface.co/datasets/kuznetsoffandrey/sberquad) для решения задачи генерации ответа на вопрос по контексту. Только будем вместо этого подбирать контекст к вопросу.

In [2]:
from datasets import load_dataset
import pandas as pd
from tqdm.auto import tqdm

tqdm.pandas()

In [3]:
ds = load_dataset("kuznetsoffandrey/sberquad", split='train[:1000]')
df = pd.DataFrame(ds)
df.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,id,title,context,question,answers
0,62310,SberChallenge,В протерозойских отложениях органические остат...,чем представлены органические остатки?,{'text': ['известковыми выделениями сине-зелён...
1,28101,SberChallenge,В протерозойских отложениях органические остат...,что найдено в кремнистых сланцах железорудной ...,"{'text': ['нитевидные водоросли, грибные нити'..."
2,48834,SberChallenge,В протерозойских отложениях органические остат...,что встречается в протерозойских отложениях?,"{'text': ['органические остатки'], 'answer_sta..."
3,83056,SberChallenge,В протерозойских отложениях органические остат...,что относится к числу древнейших растительных ...,{'text': ['скопления графито-углистого веществ...
4,5816,SberChallenge,В протерозойских отложениях органические остат...,как образовалось графито-углистое вещество?,{'text': ['в результате разложения Corycium en...


Или так, если не хочется ставить библиотеку ради одной функции:

In [4]:
splits = {
    'train': 'sberquad/train-00000-of-00001.parquet',
    'validation': 'sberquad/validation-00000-of-00001.parquet',
    'test': 'sberquad/test-00000-of-00001.parquet'
    }
df = pd.read_parquet("hf://datasets/kuznetsoffandrey/sberquad/" + splits["train"])[:1000]
df.head()

,id,title,context,question,answers
0,62310,SberChallenge,В протерозойских отложениях органические остат...,чем представлены органические остатки?,{'text': ['известковыми выделениями сине-зелён...
1,28101,SberChallenge,В протерозойских отложениях органические остат...,что найдено в кремнистых сланцах железорудной ...,"{'text': ['нитевидные водоросли, грибные нити'..."
2,48834,SberChallenge,В протерозойских отложениях органические остат...,что встречается в протерозойских отложениях?,"{'text': ['органические остатки'], 'answer_sta..."
3,83056,SberChallenge,В протерозойских отложениях органические остат...,что относится к числу древнейших растительных ...,{'text': ['скопления графито-углистого веществ...
4,5816,SberChallenge,В протерозойских отложениях органические остат...,как образовалось графито-углистое вещество?,{'text': ['в результате разложения Corycium en...


In [5]:
df.shape

(1000, 5)

Теперь переделаем датасет в тот вид, который нам удобен:
- избавимся от ненужных столбцов (айди, название, ответы)
- удалим дубликаты
- придумаем целевую переменную
- добавим негативы в данные

Начнем с удаления дубликатов. Заметим, что сами по себе запросы и тексты могут повторятся, а вот повтор пар - это уже нехорошо. Но спойлер: запросы не повторяются

In [6]:
df = df.drop_duplicates(['context', 'question'], ignore_index=True)

In [7]:
df.value_counts('context').head(2)

,count
context,
"Город Байконур и космодром Байконур вместе образуют комплекс Байконур , арендованный Россией у Казахстана на период до 2050 года. Эксплуатация космодрома стоит около 9 млрд рублей в год (стоимость аренды комплекса Байконур составляет 115 млн долларов — около 7,4 млрд рублей в год; ещё около 1,5 млрд рублей в год Россия тратит на поддержание объектов космодрома), что составляет 4,2 % от общего бюджета Роскосмоса на 2012 год. Кроме того, из федерального бюджета России в бюджет города Байконура ежегодно осуществляется безвозмездное поступление в размере 1,16 млрд рублей (по состоянию на 2012 год). В общей сложности космодром и город обходятся бюджету России в 10,16 млрд рублей в год.",18
"В целях совершенствования договорно-правовой базы, обеспечивающей эффективное сотрудничество при эксплуатации космодрома Байконур, создания необходимых условий для жизнеобеспечения персонала комплекса, проживающего в г. Байконур, 15 июня 2012 года президенты России и Казахстана договорились о воссоздании Российско-Казахстанской межправительственной комиссии по комплексу Байконур . Такая комиссия была создана постановлением правительства РФ от 13 декабря 2012 г. № 1301, председателем комиссии назначен первый заместитель председателя правительства РФ И. И. Шувалов.",16


In [8]:
df.value_counts('question').head(2)

,count
question,
Анализ чего привел Кеплера к догадкам о плотной упаковке шаров?,1
От чего обязательно перестает зависеть величина постоянных издержек?,1


Теперь удалим лишние колонки

In [9]:
df = df.drop(columns=['id', 'title', 'answers'])
df.head(2)

,context,question
0,В протерозойских отложениях органические остат...,чем представлены органические остатки?
1,В протерозойских отложениях органические остат...,что найдено в кремнистых сланцах железорудной ...


Пусть целевая переменная будет бинарной: 1, если в изначальном датасете такая пара была, и 0, если нет. Очевидно, что изначально у нас есть только 1, негативные пары нам надо будет сгенерировать

In [10]:
df['target'] = 1

Займемся негативами: сгенерируем каждому тексту столько негативов, сколько у него уже есть позитивов (либо столько, сколько сможем). Как думаете, какие более умные способы генерации, чем просто рандом, можно предложить?

In [11]:
SEED = 42

stat = df.value_counts('context')

In [12]:
# функция достаточно медленная, для всего трейна будет работать минут 10;
# можете попробовать ускорить
def get_negatives(text):
    n = stat.loc[text]
    return df[df.context != text].question.sample(n,
                                                  random_state=SEED).tolist()

In [13]:
negs = pd.DataFrame()
negs['context'] = df.context.copy()
negs['question'] = df.context.progress_apply(get_negatives)
negs = negs.explode('question').drop_duplicates()
negs['target'] = 0
print(negs.shape)
negs.head(2)

  0%|          | 0/1000 [00:00<?, ?it/s]

(1000, 3)


,context,question,target
0,В протерозойских отложениях органические остат...,Как расположены внутренние органы у змей?,0
0,В протерозойских отложениях органические остат...,В какой части государства эта технология не по...,0


Соберем все вместе и вот наши финальные данные

In [14]:
data = pd.concat([df, negs], ignore_index=True)
data.head()

,context,question,target
0,В протерозойских отложениях органические остат...,чем представлены органические остатки?,1
1,В протерозойских отложениях органические остат...,что найдено в кремнистых сланцах железорудной ...,1
2,В протерозойских отложениях органические остат...,что встречается в протерозойских отложениях?,1
3,В протерозойских отложениях органические остат...,что относится к числу древнейших растительных ...,1
4,В протерозойских отложениях органические остат...,как образовалось графито-углистое вещество?,1


## TF-IDF и евклидово расстояние

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances

In [16]:
vectorizer = TfidfVectorizer()
context = vectorizer.fit_transform(data.context[:1000])
question = vectorizer.transform(data.question[:1000])

print(context.shape, question.shape)

(1000, 8316) (1000, 8316)


In [17]:
dst = euclidean_distances(context, question)
dst.shape

(1000, 1000)

In [18]:
top = dst.argsort(axis=1)[:, :5]
top

array([[654,   1,   3,   0,   2],
       [654,   1,   3,   0,   2],
       [654,   1,   3,   0,   2],
       ...,
       [654, 994, 996, 997, 998],
       [654, 994, 996, 997, 998],
       [654, 999, 521, 889, 252]])

In [19]:
q = 0

print(data.loc[q, 'context'])
data.loc[top[q], 'question']

В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.


,question
654,Кого учила шить жена Мартина Каппы？
1,что найдено в кремнистых сланцах железорудной ...
3,что относится к числу древнейших растительных ...
0,чем представлены органические остатки?
2,что встречается в протерозойских отложениях?


In [20]:
q = 10

print(data.loc[q, 'context'])
data.loc[top[q], 'question']

Город Байконур и космодром Байконур вместе образуют комплекс Байконур , арендованный Россией у Казахстана на период до 2050 года. Эксплуатация космодрома стоит около 9 млрд рублей в год (стоимость аренды комплекса Байконур составляет 115 млн долларов — около 7,4 млрд рублей в год; ещё около 1,5 млрд рублей в год Россия тратит на поддержание объектов космодрома), что составляет 4,2 % от общего бюджета Роскосмоса на 2012 год. Кроме того, из федерального бюджета России в бюджет города Байконура ежегодно осуществляется безвозмездное поступление в размере 1,16 млрд рублей (по состоянию на 2012 год). В общей сложности космодром и город обходятся бюджету России в 10,16 млрд рублей в год.


,question
654,Кого учила шить жена Мартина Каппы？
13,Что образуют вместе город Байконур и космодром...
18,В какую сумму обходится эксплуатация космодром...
23,Во сколько обходится содержание и эксплуатация...
14,Какая сумма выплачивается городу Байконур из ф...


## TF-Idf и обючаемая мера расстояния

Будем учиться предсказывать число, таргет, по парам векторов.

In [21]:
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split

In [22]:
vectorizer = TfidfVectorizer()
context = vectorizer.fit_transform(data.context)
question = vectorizer.transform(data.question)

print(context.shape, question.shape)

(2000, 8316) (2000, 8316)


In [23]:
embs = np.hstack([context.toarray(), question.toarray()])
print(embs.shape)

(2000, 16632)


Поделим на трейн и тест

In [24]:
x_train, x_test, y_train, y_test = train_test_split(embs, data.target,
                                                    test_size=0.25,
                                                    random_state=42)

Сначала линейная регрессия

In [25]:
rgn = LinearRegression()
rgn.fit(x_train, y_train)
rgn.score(x_train, y_train), rgn.score(x_test, y_test)

(0.9246733366316273, -1.969691356486672e+24)

Получилась ерунда, но это ожидаемо. Побробуем логистическую регресию. Почему так вообще можно?

In [26]:
clf = LogisticRegression(random_state=0)
clf.fit(x_train, y_train)
clf.score(x_train, y_train), clf.score(x_test, y_test)

(0.9593333333333334, 0.92)

А вот здесь как будто и неплохо. Посмотрим на примеры

In [27]:
clf.predict_proba([embs[10]]), clf.predict_proba([embs[500]]), \
clf.predict_proba([embs[1500]]), clf.predict_proba([embs[1950]])

(array([[0.35741924, 0.64258076]]),
 array([[0.13796119, 0.86203881]]),
 array([[0.74965022, 0.25034978]]),
 array([[0.87928873, 0.12071127]]))

In [28]:
data.loc[[10, 500, 1500, 1950]].style

,context,question,target
10,"Город Байконур и космодром Байконур вместе образуют комплекс Байконур , арендованный Россией у Казахстана на период до 2050 года. Эксплуатация космодрома стоит около 9 млрд рублей в год (стоимость аренды комплекса Байконур составляет 115 млн долларов — около 7,4 млрд рублей в год; ещё около 1,5 млрд рублей в год Россия тратит на поддержание объектов космодрома), что составляет 4,2 % от общего бюджета Роскосмоса на 2012 год. Кроме того, из федерального бюджета России в бюджет города Байконура ежегодно осуществляется безвозмездное поступление в размере 1,16 млрд рублей (по состоянию на 2012 год). В общей сложности космодром и город обходятся бюджету России в 10,16 млрд рублей в год.",В каком году истекает договор аренды Байконура,1
500,"В Миссолонги Байрон заболел лихорадкой, продолжая отдавать все свои силы на борьбу за свободу страны. 19 января 1824 года он писал Хэнкопу: Мы готовимся к экспедиции , а 22 января, в день своего рождения, он вошёл в комнату полковника Стенхопа, где было несколько человек гостей, и весело сказал: Вы упрекаете меня, что я не пишу стихов, а вот я только что написал стихотворение . И Байрон прочёл: Сегодня мне исполнилось 36 лет . Постоянно хворавшего Байрона очень тревожила болезнь его дочери Ады. Получив письмо с хорошей вестью о её выздоровлении, он захотел выехать прогуляться с графом Гамба. Во время прогулки пошёл страшный дождь, и Байрон окончательно захворал. Последними словами поэта были отрывочные фразы: Сестра моя! дитя моё!.. бедная Греция!.. я отдал ей время, состояние, здоровье!.. теперь отдаю ей и жизнь! .",Что тревожило постоянно хворавшего Байрона?,1
1500,"В Миссолонги Байрон заболел лихорадкой, продолжая отдавать все свои силы на борьбу за свободу страны. 19 января 1824 года он писал Хэнкопу: Мы готовимся к экспедиции , а 22 января, в день своего рождения, он вошёл в комнату полковника Стенхопа, где было несколько человек гостей, и весело сказал: Вы упрекаете меня, что я не пишу стихов, а вот я только что написал стихотворение . И Байрон прочёл: Сегодня мне исполнилось 36 лет . Постоянно хворавшего Байрона очень тревожила болезнь его дочери Ады. Получив письмо с хорошей вестью о её выздоровлении, он захотел выехать прогуляться с графом Гамба. Во время прогулки пошёл страшный дождь, и Байрон окончательно захворал. Последними словами поэта были отрывочные фразы: Сестра моя! дитя моё!.. бедная Греция!.. я отдал ей время, состояние, здоровье!.. теперь отдаю ей и жизнь! .",Когда Франциск I Французский завоевал Миланское герцогство?,0
1950,"Первый же матч стал провалом — сборная Нидерландов буквально уничтожила итальянцев со счётом 3:0. Во втором матче только старания Джанлуиджи Буффона спасли итальянцев от поражения — ничья 1:1 и незабитый румыном Адрианом Муту пенальти. В третьем матче сборная Италии сумела взять верх над Францией со счётом 2:0 и выйти в плей-офф, где в четвертьфинале проиграла Испании по пенальти со счётом 2:4, причём в основное время у итальянцев было больше шансов открыть счёт, хотя команды играли от обороны. По решению руководства федерации Донадони был уволен — хотя с ним велись переговоры о продолжении работы до 2010 года, чем были недовольны болельщики.",Что уцелело в городе Мец из старинных укреплений?,0
